In [1]:
# https://chatgpt.com/share/d42a16ed-1edd-4092-b7ce-74a9fd2d901d

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
import itertools

c:\Users\Alessandro\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\Alessandro\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\Alessandro\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
c:\Users\Alessandro\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [3]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [4]:
dati = pd.read_csv('Bitcoin (€) da yfinance dal 17-09-2014 al 24-04-2024.csv')
dati = dati[['Date', 'Close']]
dati = dati.rename(columns = {'Date':'Timestamp', 'Close': 'Price'})
dati['Timestamp'] = pd.to_datetime(dati['Timestamp'])
dati['Timestamp'] = dati['Timestamp'].dt.strftime('%Y-%m-%d')
dati['Timestamp'] = pd.to_datetime(dati['Timestamp'], format='%Y-%m-%d')
dati

,Timestamp,Price
0,2014-09-17,355.957367
1,2014-09-18,328.539368
2,2014-09-19,307.761139
3,2014-09-20,318.758972
4,2014-09-21,310.632446
...,...,...
3503,2024-04-20,59876.710938
3504,2024-04-21,60956.074219
3505,2024-04-22,60919.242188
3506,2024-04-23,62729.296875


In [5]:
# dati = pd.read_csv('Bitcoin (€) da yfinance dal 06-05-2022 alle 21 al 05-05-2024 alle 19 con granularità oraria.csv')
# dati = dati[['Datetime', 'Close']]
# dati = dati.rename(columns = {'Datetime':'Timestamp', 'Close': 'Price'})
# dati['Timestamp'] = pd.to_datetime(dati['Timestamp'])
# dati['Timestamp'] = dati['Timestamp'].dt.strftime('%Y-%m-%d %H')
# dati['Timestamp'] = pd.to_datetime(dati['Timestamp'], format='%Y-%m-%d %H')
# dati

In [6]:
data_inizio = '2020-02-29'
data_fine = '2020-09-01'
indice_inizio = dati[dati['Timestamp'] == data_inizio].index[0]
indice_fine = dati[dati['Timestamp'] == data_fine].index[0]
dati = dati[indice_inizio:indice_fine + 1].reset_index(drop=True)
dati

,Timestamp,Price
0,2020-02-29,7798.025879
1,2020-03-01,7748.164551
2,2020-03-02,7959.358398
3,2020-03-03,7862.485352
4,2020-03-04,7861.055176
...,...,...
181,2020-08-28,9695.468750
182,2020-08-29,9664.351562
183,2020-08-30,9824.922852
184,2020-08-31,9785.023438


In [7]:
# data_inizio = '2023-06-01 00'
# data_fine = '2023-12-01 00'
# indice_inizio = dati[dati['Timestamp'] == data_inizio].index[0]
# indice_fine = dati[dati['Timestamp'] == data_fine].index[0]
# dati = dati[indice_inizio:indice_fine + 1].reset_index(drop=True)
# dati

### Eventuale cambio dati

In [8]:
def calcola_prezzo(t):
    return dati['Price'][t] #1000 - t + 100 * math.sin(2*math.pi * t / 4)

In [9]:
dati['Price'] = dati.index.map(calcola_prezzo)
dati

,Timestamp,Price
0,2020-02-29,7798.025879
1,2020-03-01,7748.164551
2,2020-03-02,7959.358398
3,2020-03-03,7862.485352
4,2020-03-04,7861.055176
...,...,...
181,2020-08-28,9695.468750
182,2020-08-29,9664.351562
183,2020-08-30,9824.922852
184,2020-08-31,9785.023438


In [10]:
def calcola_percentuale_acquisto(prezzi, giorni, min_giorni_passati, denominatore_max_drawdown, 
                                 perc_acquisto_fisso):
    if len(prezzi) < min_giorni_passati:
        return perc_acquisto_fisso
    
    ultimi_giorni = prezzi[-giorni:]
    massimo = max(ultimi_giorni)
    minimo = min(ultimi_giorni)
    discesa_attuale = (ultimi_giorni.iloc[-2] - ultimi_giorni.iloc[-1]) / ultimi_giorni.iloc[-2]

    if denominatore_max_drawdown:
        max_drawdown = (massimo - minimo) / massimo
        if max_drawdown == 0:
            return perc_acquisto_fisso
        percentuale = discesa_attuale / max_drawdown
    else:
        max_discesa_due_giorni = max(
            [(ultimi_giorni.iloc[i] - ultimi_giorni.iloc[i + 1]) / ultimi_giorni.iloc[i] for i in range(len(ultimi_giorni) - 1)]
        )
        if max_discesa_due_giorni == 0:
            return perc_acquisto_fisso
        percentuale = discesa_attuale / max_discesa_due_giorni
    
    return min(1, percentuale)

def calcola_percentuale_vendita(prezzi, giorni, min_giorni_passati, denominatore_max_drawdown, 
                                perc_vendita_fisso):
    if len(prezzi) < min_giorni_passati:
        return perc_vendita_fisso
    
    ultimi_giorni = prezzi[-giorni:]
    massimo = max(ultimi_giorni)
    minimo = min(ultimi_giorni)
    salita_attuale = (ultimi_giorni.iloc[-1] - ultimi_giorni.iloc[-2]) / ultimi_giorni.iloc[-2]

    if denominatore_max_drawdown:
        max_drawdown = (massimo - minimo) / massimo
        if max_drawdown == 0:
            return perc_vendita_fisso
        percentuale = salita_attuale / max_drawdown
    else:
        max_salita_due_giorni = max(
            [(ultimi_giorni.iloc[i + 1] - ultimi_giorni.iloc[i]) / ultimi_giorni.iloc[i] for i in range(len(ultimi_giorni) - 1)]
        )
        if max_salita_due_giorni == 0:
            return perc_vendita_fisso
        percentuale = salita_attuale / max_salita_due_giorni
    
    return min(1, percentuale)

def strategia_trading(dati, giorni_acquisto, min_giorni_passati_acquisto, denominatore_acquisto_max_drawdown, 
                      usa_liquidita_iniziale, giorni_vendita, min_giorni_passati_vendita, 
                      denominatore_vendita_max_drawdown, liquidita_iniziale, token_iniziali,
                      perc_acquisto_fisso, perc_vendita_fisso, min_acquisto, min_vendita,
                      perc_commissione_acquisto, perc_commissione_vendita):

    liquidita = liquidita_iniziale
    token = token_iniziali
    portafoglio = []

    for i in range(1, len(dati)):
        prezzo_oggi = dati.iloc[i]
        prezzo_ieri = dati.iloc[i-1]
        
        # Decidere se comprare o vendere
        if prezzo_oggi < prezzo_ieri:
            percentuale_acquisto = calcola_percentuale_acquisto(
                dati.iloc[:i+1], giorni_acquisto, min_giorni_passati_acquisto, 
                denominatore_acquisto_max_drawdown, perc_acquisto_fisso
            )
            if usa_liquidita_iniziale:
                da_acquistare = liquidita_iniziale * percentuale_acquisto / prezzo_oggi
            else:
                da_acquistare = liquidita * percentuale_acquisto / prezzo_oggi
            
            costo_iniziale = da_acquistare * prezzo_oggi
            
            # Controllo se il costo iniziale senza commissioni supera la soglia minima di acquisto
            if costo_iniziale >= min_acquisto:
                costo_totale = costo_iniziale * (1 + perc_commissione_acquisto)
                if liquidita >= costo_totale:
                    token += da_acquistare
                    liquidita -= costo_totale

        elif prezzo_oggi > prezzo_ieri:
            percentuale_vendita = calcola_percentuale_vendita(
                dati.iloc[:i+1], giorni_vendita, min_giorni_passati_vendita, 
                denominatore_vendita_max_drawdown, perc_vendita_fisso
            )
            da_vendere = token * percentuale_vendita
            ricavo_iniziale = da_vendere * prezzo_oggi
            
            # Controllo se il ricavo iniziale senza commissioni supera la soglia minima di vendita
            if ricavo_iniziale >= min_vendita:
                ricavo_totale = ricavo_iniziale * (1 - perc_commissione_vendita)
                token -= da_vendere
                liquidita += ricavo_totale
        
        # Calcolo del valore del portafoglio
        valore_portafoglio = liquidita + token * prezzo_oggi
        portafoglio.append(valore_portafoglio)
    
    return portafoglio

In [11]:
liquidita_iniziale = 1000
token_iniziali = 0

# Configurazione delle variabili
giorni_acquisto = 30
min_giorni_passati_acquisto = 7
denominatore_acquisto_max_drawdown = False
usa_liquidita_iniziale = True
perc_acquisto_fisso = 0.01  # Percentuale fissa di acquisto
perc_commissione_acquisto = 0.001
min_acquisto = 1  # Importo minimo per effettuare un acquisto in euro

giorni_vendita = 30
min_giorni_passati_vendita = 7
denominatore_vendita_max_drawdown = False
perc_vendita_fisso = 0.01  # Percentuale fissa di vendita
perc_commissione_vendita = 0.001
min_vendita = 1  # Importo minimo per effettuare una vendita in euro

# Calcola il portafoglio giorno per giorno
valori_portafoglio = strategia_trading(dati['Price'], giorni_acquisto, min_giorni_passati_acquisto, 
                                       denominatore_acquisto_max_drawdown, usa_liquidita_iniziale, 
                                       giorni_vendita, min_giorni_passati_vendita, 
                                       denominatore_vendita_max_drawdown, 
                                       liquidita_iniziale, token_iniziali,
                                       perc_acquisto_fisso, perc_vendita_fisso, 
                                       min_acquisto, min_vendita, 
                                       perc_commissione_acquisto, perc_commissione_vendita)

valori_portafoglio

[999.99,
 1000.2625727445153,
 1000.1275456542411,
 1000.113880842091,
 1000.9909690394732,
 1000.8923834377131,
 998.5018802423142,
 988.6895751127263,
 986.3053357557515,
 988.0023983009626,
 989.3370059431325,
 906.7204156494404,
 924.1506498810186,
 923.9721935612362,
 930.3632282350586,
 920.9191872082262,
 938.3718269753003,
 939.7166057463952,
 976.4015921737125,
 976.3788034870508,
 976.3200183113657,
 974.6952201541928,
 990.1900866629351,
 994.9458651538639,
 993.5334410847955,
 992.3412103427364,
 985.1020548585489,
 975.3966677884682,
 957.6944107042515,
 1003.4135046677978,
 1003.8587809393152,
 1013.2041705646669,
 1022.7378526612054,
 1021.624147354666,
 1026.0692717427132,
 1023.6959985336987,
 1040.4729682901855,
 1036.7455359366263,
 1042.3793509225413,
 1040.273800181083,
 1026.3616071238685,
 1026.16857665984,
 1029.948006015895,
 1026.0389135203461,
 1023.1666068177733,
 1010.6020506687976,
 1078.9259890986195,
 1076.1707906361885,
 1091.7431457381015,
 1086.939161

In [13]:
valori_portafoglio[-1] / valori_portafoglio[0]

1.2650861913134097